In [1]:
import pytraj as pt
import numpy as np
import pandas as pd

import copy
import gc

In [2]:
strucDir='structures'
trajName='traj.1us.dcd'
trajPath='/'.join([strucDir,trajName])
topName='topol.parm7'
topPath='/'.join([strucDir,topName])
iterTraj=pt.iterload(trajPath,top=topPath)
iterTraj

pytraj.TrajectoryIterator, 8335 frames: 
Size: 13.000000 (GB)
<Topology: 70296 atoms, 4254 residues, 12 mols, PBC with box type = ortho>
           

In [4]:
channelResids=np.arange(1322,1371)
nRes=4254
nChains=3
resPerChain=nRes/nChains
resList=np.concatenate([
    channelResids + resPerChain*iRes \
    for iRes in np.arange(nChains)
])
resList

array([1322, 1323, 1324, ..., 4204, 4205, 4206])

In [5]:
commandList=[
    "vector center :%g"%iRes \
    for iRes in resList
]
comData=pt.compute(commandList,iterTraj).values()
comData[0]

array([[ 87.32514997, 113.50049432, 104.41338308],
       [ 86.77643345, 114.67273315, 104.50121851],
       [ 86.13430181, 114.50329489, 104.03314161],
       ...,
       [ 76.95923399, 120.91999186,  95.6058307 ],
       [ 77.84140984, 119.96407103,  95.48467719],
       [ 78.08829844, 120.27425519,  94.9651428 ]])

In [6]:
comData=np.array(comData)
comData.shape

(147, 8335, 3)

In [7]:
import tqdm

In [9]:
comFrameList=[]
for iRes,resData in enumerate(
    tqdm.tqdm(comData,desc='compiling data frame')):
    comFrame=pd.DataFrame({
        'Resid':[resList[iRes]]*len(resData),
        'Frame':np.arange(len(resData)),
        'X':resData[:,0],
        'Y':resData[:,1],
        'Z':resData[:,2]
    })
    comFrameList.append(comFrame)
    
comDataFrame=pd.concat(comFrameList)
comFrameList=[]
comDataFrame.to_csv('channel_COM_data.csv')
comDataFrame.head()

compiling data frame: 100%|██████████| 147/147 [00:00<00:00, 275.73it/s]


,Frame,Resid,X,Y,Z
0,0,1322,87.325150,113.500494,104.413383
1,1,1322,86.776433,114.672733,104.501219
2,2,1322,86.134302,114.503295,104.033142
3,3,1322,84.937330,114.524715,104.247451
4,4,1322,87.220211,114.127925,104.109741
